# Análisis Exploratorio

TODO
- [x] Comience describiendo cuantas variables y observaciones tiene disponibles, el tipo de cada una de las variables.
- [ ] Haga un resumen de las variables numéricas e investigue si siguen una distribución normal y tablas de frecuencia para las variables categóricas, escriba lo que vaya encontrando.
- [ ] Encoding de las variables categóricas.
- [ ] Arreglar estado_pil
- [ ] Cruce las variables que considere que son las más importantes para hallar los elementos clave que lo pueden llevar a comprender lo que está causando el problema encontrado.
- [ ] Haga gráficos exploratorios que le de ideas del estado de los datos.
- [ ] Haga un agrupamiento (clustering) e interprete los resultados. 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# Load the data
df_2011 = pd.read_csv('data/2011.csv')
df_2013 = pd.read_csv('data/2013.csv')
df_2014 = pd.read_csv('data/2014.csv')
df_2016 = pd.read_csv('data/2016.csv')
df_2017 = pd.read_csv('data/2017.csv')
df_2018 = pd.read_csv('data/2018.csv')
df_2019 = pd.read_csv('data/2019.csv')
df_2020 = pd.read_csv('data/2020.csv')
df_2022 = pd.read_csv('data/2022.csv')

In [3]:
# Concatenate the data with only the columns that are present in the df_2011
df = pd.concat([
    df_2011,
    df_2013,
    df_2014,
    df_2016,
    df_2017,
    df_2018,
    df_2019,
    df_2020,
    df_2022
], axis=0, ignore_index=True)

In [4]:
df.head()

,dia_ocu,mes_ocu,dia_sem_ocu,hora_ocu,depto_ocu,zona_ocu,sexo_pil,edad_pil,mayor_menor,estado_pil,tipo_vehi,marca_vehi,causa_acc,year
0,27,Junio,Lunes,17,Guatemala,10,Hombre,42,Mayor,Normal,Automovil,Audi,Colisión,2011
1,18,Septiembre,Domingo,0,Guatemala,7,Hombre,25,Mayor,Ebrio,Automovil,Audi,Empotro,2011
2,5,Octubre,Miercoles,19,Quetzaltenango,Ignorado,Hombre,26,Mayor,Ebrio,Automovil,Audi,Colisión,2011
3,15,Febrero,Martes,12,Izabal,Ignorado,Hombre,59,Mayor,Ignorado,Camioneta,Land rover,Colisión,2011
4,17,Junio,Viernes,18,Guatemala,11,Mujer,25,Mayor,Normal,Camioneta,Land rover,Colisión,2011


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76792 entries, 0 to 76791
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   dia_ocu      76792 non-null  int64 
 1   mes_ocu      76792 non-null  object
 2   dia_sem_ocu  76792 non-null  object
 3   hora_ocu     76792 non-null  object
 4   depto_ocu    76792 non-null  object
 5   zona_ocu     76792 non-null  object
 6   sexo_pil     76792 non-null  object
 7   edad_pil     76792 non-null  object
 8   mayor_menor  76792 non-null  object
 9   estado_pil   76792 non-null  object
 10  tipo_vehi    76792 non-null  object
 11  marca_vehi   76792 non-null  object
 12  causa_acc    76792 non-null  object
 13  year         76792 non-null  int64 
dtypes: int64(2), object(12)
memory usage: 8.2+ MB


In [6]:
# Separate numerical and categorical columns
numerical = df.select_dtypes(include=['int64', 'float64']).columns
categorical = df.select_dtypes(include=['object']).columns

In [7]:
# Numerical NA's
df[numerical].isna().sum()

dia_ocu    0
year       0
dtype: int64

Dada la naturaleza con la que la información fue recolectada, los datos no son consistentes en cuanto a sus mayúsculas y minúsculas. Tampoco se tienen valores nulos, si no que aparecen como "ignorado", por lo que es necesario su tratamiento.

In [8]:
# To lower Case
df[categorical] = df[categorical].apply(lambda x: x.str.lower())

In [9]:
# Categorical NA's
df[categorical].isna().sum()

mes_ocu            0
dia_sem_ocu        0
hora_ocu       56008
depto_ocu          0
zona_ocu           0
sexo_pil           0
edad_pil           0
mayor_menor        0
estado_pil         0
tipo_vehi          0
marca_vehi         0
causa_acc          0
dtype: int64

## Análisis de las variables

In [10]:
# Categorical variables
for col in categorical:
    # Print them sorted
    print(df[col].value_counts().sort_index())

mes_ocu
abril         6583
agosto        6484
diciembre     6612
enero         6646
febrero       6603
julio         6330
junio         6206
marzo         7090
mayo          6466
noviembre     5841
octubre       5500
septiembre    5767
septiemre      664
Name: count, dtype: int64
dia_sem_ocu
domingo      13992
jueves        9648
lunes        10108
martes        8836
miercoles     3629
miércoles     5309
sabado        5557
sábado        8780
viernes      10933
Name: count, dtype: int64
hora_ocu
0           1098
1            714
10           662
11           774
12           719
13           759
14           863
15           997
16           912
17          1189
18          1261
19          1463
2            527
20          1437
21          1414
22          1254
23          1115
3            408
4            291
5            414
6            543
7            702
8            645
9            608
ignorada      15
Name: count, dtype: int64
depto_ocu
alta verapaz       2933
alta verpaz     

Muchos datos están inconsistentes por culpa de las tíldes, por lo que se procede a corregirlos.

In [11]:
accented_vowels = {
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'Á': 'A',
    'É': 'E',
    'Í': 'I',
    'Ó': 'O',
    'Ú': 'U'
}

df.replace(accented_vowels, regex=True, inplace=True)

# Replace all 'ignorada' and 'ignorado' with 'NA' in the categorical columns
df[categorical] = df[categorical].replace(['ignorada', 'ignorado'], 'NA')

También algunos están mal escritos

In [12]:
df['depto_ocu'] = df['depto_ocu'].replace('alta verpaz', 'alta verapaz')
df['mes_ocu'] = df['mes_ocu'].replace('septiemre', 'septiembre')
df['causa_acc'] = df['causa_acc'].replace('volco', 'vuelco')
# df['causa_acc'] = df['causa_acc'].replace('choque', 'colision')

Existen algunos valores en _estado\_pil_ con valor numérico. En base al diccionario de variables proveído por el INE, son cambiados

In [16]:
df['estado_pil'] = df['estado_pil'].replace(['1', '2', '3', '4', '5', '6', '7', '8', '9'], 'ebrio')
df['estado_pil'] = df['estado_pil'].replace(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19'], 'no ebrio')
df['estado_pil'] = df['estado_pil'].replace('normal', 'no ebrio')

In [17]:
# Categorical variables
for col in categorical:
    # Print them sorted
    print(df[col].value_counts().sort_index())

mes_ocu
abril         6583
agosto        6484
diciembre     6612
enero         6646
febrero       6603
julio         6330
junio         6206
marzo         7090
mayo          6466
noviembre     5841
octubre       5500
septiembre    6431
Name: count, dtype: int64
dia_sem_ocu
domingo      13992
jueves        9648
lunes        10108
martes        8836
miercoles     8938
sabado       14337
viernes      10933
Name: count, dtype: int64
hora_ocu
-1     56023
 0      1098
 1       714
 2       527
 3       408
 4       291
 5       414
 6       543
 7       702
 8       645
 9       608
 10      662
 11      774
 12      719
 13      759
 14      863
 15      997
 16      912
 17     1189
 18     1261
 19     1463
 20     1437
 21     1414
 22     1254
 23     1115
Name: count, dtype: int64
depto_ocu
alta verapaz       3006
baja verapaz       1328
chimaltenango      2866
chiquimula         1652
el progreso        1780
escuintla          6055
guatemala         30020
huehuetenango      1546
izaba

Arreglando los valores nulos, se convierten en columnas numéricas

In [15]:
df['hora_ocu'] = df['hora_ocu'].replace('NA', None).fillna(-1).astype(int)
df['zona_ocu'] = df['zona_ocu'].replace('NA', None).fillna(-1).astype(int)
df['edad_pil'] = df['edad_pil'].replace('NA', None).fillna(-1).astype(int)

numerical = numerical.append(pd.Index(['hora_ocu', 'zona_ocu', 'edad_pil']))